In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

torch.manual_seed(123)
np.random.seed(123)

##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

from google.colab import drive
drive.mount('/content/drive')\

     |████████████████████████████████| 46.9MB 114kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 4.3MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595697 sha256=a222a53b4fd1ecce521b0fb879f438da43b8d924de41e66b736e1b03eaf3e56b
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 4.2MB/s 
     |████████████████████████████████| 757kB 5.3MB/s 
     |████████████████████████████████| 901kB 38.6MB/s 
     |████████████████████████████████| 3.0MB 21.6MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-vozj886i
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-vozj886i
  Created wheel for kobert

In [2]:
# 학습용 데이터셋 불러오기
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/Dacon/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Dacon/test.csv')

# 데이터 전처리
train=train[['과제명', '요약문_연구내용','label']]
test=test[['과제명', '요약문_연구내용']]
train['요약문_연구내용'].fillna('NAN', inplace=True)
test['요약문_연구내용'].fillna('NAN', inplace=True)

# 대표 질병 5개 추출
train['data']=train['과제명']+train['요약문_연구내용']
test['data']=test['과제명']+test['요약문_연구내용']

def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean

# Train / Test set 분리
from sklearn.model_selection import train_test_split
trains, tests = train_test_split(train, test_size=0.2, random_state=123)
print("train shape is:", len(trains))
print("test shape is:", len(tests))


# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([clean_text(i)]) for i in dataset['data']]
        self.labels = [np.int32(i) for i in dataset['label']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
        
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate = 3e-5


data_train = BERTDataset(trains, 3, 2, tok, max_len, True, False)
data_test = BERTDataset(tests, 3, 2, tok, max_len, True, False)

# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

train shape is: 139443
test shape is: 34861
using cached model


In [3]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 46, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  

In [5]:
highest_acc = 0
patience = 0

# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    #테스트 문장 예측
    
    model.eval() # 평가 모드로 변경
    
    for test_batch_id, (test_token_ids, test_valid_length, test_segment_ids, test_label) in enumerate(tqdm_notebook(test_dataloader)):
        test_token_ids = test_token_ids.long().to(device)
        test_segment_ids = test_segment_ids.long().to(device)z
        test_valid_length= test_valid_length
        test_label = test_label.long().to(device)
        test_out = model(token_ids, valid_length, segment_ids)
        test_loss = loss_fn(out, label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (test_batch_id+1)))

    if test_acc > highest_acc:
        torch.save({
            'epoch': e,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': test_loss,
            }, '/content/drive/MyDrive/Dacon/torchckpt/model.pt')
        patience = 0
    else:
        print("test acc did not improved. best:{} current:{}".format(highest_acc, test_acc))
        patience += 1
        if patience > 5:
            break
    print('current patience: {}'.format(patience))
    print("************************************************************************************")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


epoch 1 batch id 1 loss 3.746753692626953 train acc 0.046875
epoch 1 batch id 201 loss 2.833305597305298 train acc 0.35152363184079605
epoch 1 batch id 401 loss 1.6726614236831665 train acc 0.5844373441396509
epoch 1 batch id 601 loss 0.941444456577301 train acc 0.6603057404326124
epoch 1 batch id 801 loss 1.0692002773284912 train acc 0.7006476279650437
epoch 1 batch id 1001 loss 1.3497064113616943 train acc 0.7236357392607392
epoch 1 batch id 1201 loss 0.9132053852081299 train acc 0.7399432764363031
epoch 1 batch id 1401 loss 1.078210711479187 train acc 0.7513048715203426
epoch 1 batch id 1601 loss 0.8389707207679749 train acc 0.7601401467832605
epoch 1 batch id 1801 loss 0.8923542499542236 train acc 0.7669610632981677
epoch 1 batch id 2001 loss 0.8088604807853699 train acc 0.7727932908545727

epoch 1 train acc 0.7774260375329572


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.9019607843137151


epoch 2 batch id 1 loss 0.7161873579025269 train acc 0.8125
epoch 2 batch id 201 loss 0.6478219032287598 train acc 0.8365205223880597
epoch 2 batch id 401 loss 0.8538629412651062 train acc 0.8398924563591023
epoch 2 batch id 601 loss 0.5060431957244873 train acc 0.8388103161397671
epoch 2 batch id 801 loss 0.5608747005462646 train acc 0.841916354556804
epoch 2 batch id 1001 loss 0.7570269107818604 train acc 0.8438904845154845
epoch 2 batch id 1201 loss 0.5110994577407837 train acc 0.8461438384679434
epoch 2 batch id 1401 loss 0.6903754472732544 train acc 0.8468504639543183
epoch 2 batch id 1601 loss 0.576520562171936 train acc 0.8486492816989382
epoch 2 batch id 1801 loss 0.5613363981246948 train acc 0.8500225569128262
epoch 2 batch id 2001 loss 0.45418351888656616 train acc 0.8513633808095952

epoch 2 train acc 0.8523565518451529


...


epoch 28 test acc 1.0


epoch 29 batch id 1 loss 0.0007039100164547563 train acc 1.0
epoch 29 batch id 201 loss 0.01764519326388836 train acc 0.9936256218905473
epoch 29 batch id 401 loss 0.000551644538063556 train acc 0.9933759351620948
epoch 29 batch id 601 loss 0.008981674909591675 train acc 0.9930584442595674
epoch 29 batch id 801 loss 0.0018086571944877505 train acc 0.9934456928838952
epoch 29 batch id 1001 loss 0.0005105070886202157 train acc 0.9936938061938062
epoch 29 batch id 1201 loss 0.012225037440657616 train acc 0.9937291840133222
epoch 29 batch id 1401 loss 0.06413360685110092 train acc 0.9939440578158458
epoch 29 batch id 1601 loss 0.02267826348543167 train acc 0.9939393347907558
epoch 29 batch id 1801 loss 0.004432546440511942 train acc 0.9939530122154359
epoch 29 batch id 2001 loss 0.03234993293881416 train acc 0.9939483383308346

epoch 29 train acc 0.9939479118861864



epoch 29 test acc 1.0


epoch 30 batch id 1 loss 0.0005420205998234451 train acc 1.0
epoch 30 batch id 201 loss 0.0223513413220644 train acc 0.9941697761194029
epoch 30 batch id 401 loss 0.0005109423073008657 train acc 0.9938824812967582
epoch 30 batch id 601 loss 0.0033700866624712944 train acc 0.9936304076539102
epoch 30 batch id 801 loss 0.0024389263708144426 train acc 0.9937968164794008
epoch 30 batch id 1001 loss 0.0006548871169798076 train acc 0.9938186813186813
epoch 30 batch id 1201 loss 0.01670895516872406 train acc 0.9939243338884263
epoch 30 batch id 1401 loss 0.0039558419957757 train acc 0.9941336545324768
epoch 30 batch id 1601 loss 0.01473530288785696 train acc 0.9941638038725796
epoch 30 batch id 1801 loss 0.003709059441462159 train acc 0.9941612298722932
epoch 30 batch id 2001 loss 0.013545439578592777 train acc 0.9941669790104948

epoch 30 train acc 0.9941558627810922



epoch 30 test acc 1.0


In [20]:
class BERTDataset1(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([clean_text(i)]) for i in dataset['data']]
        self.labels = [np.int32(0) for i in dataset['data']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
        #return (self.sentences[i] + (0, ))

    def __len__(self):
        return (len(self.sentences))


test_set = BERTDataset1(test, 3, 2, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=2)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

checkpoint = torch.load('/content/drive/MyDrive/Dacon/torchckpt/model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [22]:
# import torch, gc
# gc.collect()
# torch.cuda.empty_cache()

model.eval()


result = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()
      final = np.argmax(logits)
  result.append(final)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [23]:
print(result)

[0, 0, 0, 0, 1, 16, 0, 0, 0, 14, 0, 0, 23, 0, 0, 19, 0, 0, 0, 0, 27, 0, 0, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37, 0, 0, 0, 0, 0, 29, 0, 0, 0, 0, 0, 0, 34, 0, 0, 1, 0, 5, 14, 0, 0, 0, 0, 0, 0, 43, 19, 0, 0, 0, 0, 0, 0, 43, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 18, 0, 0, 0, 34, 0, 0, 0, 0, 43, 0, 0, 0, 43, 0, 0, 0, 0, 0, 0, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 27, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 19, 0, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 8, 19, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 23, 0, 19, 0, 19, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 19, 0, 0, 0, 0, 10, 0, 0, 20, 0, 0, 45, 0, 0, 0, 45, 0, 36, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 45, 2, 0, 0, 0, 14, 0, 0, 24, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 14, 0, 8, 0, 0, 18, 0, 0, 0, 0, 0, 0, 5, 0, 23, 0, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 0,

In [26]:
submission = pd.read_csv('/content/drive/MyDrive/Dacon/sample_submission.csv')

submission['label'] = np.array(result)

submission.to_csv('/content/drive/MyDrive/Dacon/kobert_baseline.csv', index=False)